# Vectorization Model of QuCT
**Author:** Siwei Tan  

**Date:** 7/4/2024

Based on paper "[QuCT: A Framework for Analyzing Quantum Circuit by Extracting Contextual and Topological Features][1]" (MICRO 2023)

[1]: https://dl.acm.org/doi/10.1145/3613424.3614274

In the current Noisy Intermediate-Scale Quantum era, quantum circuit analysis is an essential technique for designing high-performance quantum programs. Current analysis methods exhibit either accuracy limitations or high computational complexity for obtaining precise results. To reduce this tradeoff, we propose QuCT, a unified framework for extracting, analyzing, and optimizing quantum circuits. The main innovation of QuCT is to vectorize each gate with each element, quantitatively describing the degree of the interaction with neighboring gates. Extending from the vectorization model, we can develope multiple downstream models for fidelity prediction and unitary decomposition, etc. In this tutorial, we introduce the APIs of the vectorization model of QuCT.

In [1]:
%matplotlib inline

import logging
logging.basicConfig(level=logging.WARN)
import sys
sys.path.append('..')
import os
os.chdir("..")
from janusq.analysis.vectorization import RandomwalkModel, extract_device

from janusq.data_objects.random_circuit import random_circuits, random_circuit
from janusq.data_objects.backend import GridBackend

import random
import numpy as np

## Vectorization Flow
Below is the workflow to vectorize a gate in the quantum circuit. The gate is vectorized by two steps. The first step runs random walks to extract circuit features in the neighbor of the gates. the second step use a table comparison to generate the gate vector.

<div style="text-align:center;">
    <img src="../picture/2_1_feature_extraction.jpg"  width="70%" height="70%">
</div>

## Random walk
We apply random walk to extract the topological and contextual information of gates in the quantum circuit. Here is a example of random walk.

In [2]:
# generate a circuit
from janusq.analysis.vectorization import walk_from_gate

backend = GridBackend(2, 2)
circuit = random_circuit(backend, 10, .5, False)
print(circuit)

# choose a target gate
gate = random.choice(circuit.gates)

# apply random walk
paths = walk_from_gate(circuit, gate, 4, 2, backend.adjlist)

print('target gate:', gate)
print('generate paths:', paths)

     ┌───────────────────┐ ░                  ░       ░       ░       ░ »
q_0: ┤ U(2π/5,4π/5,2π/5) ├─░──────────────────░───■───░───■───░───■───░─»
     └─┬───────────────┬─┘ ░ ┌──────────────┐ ░ ┌─┴─┐ ░ ┌─┴─┐ ░   │   ░ »
q_1: ──┤ U(9π/5,π,π/5) ├───░─┤ U(2π,π,7π/5) ├─░─┤ X ├─░─┤ X ├─░───┼───░─»
     ┌─┴───────────────┴─┐ ░ └──────────────┘ ░ └───┘ ░ └───┘ ░ ┌─┴─┐ ░ »
q_2: ┤ U(3π/5,π/10,7π/5) ├─░──────────────────░───────░───────░─┤ X ├─░─»
     └┬──────────────────┤ ░                  ░       ░       ░ └───┘ ░ »
q_3: ─┤ U(7π/5,π/5,6π/5) ├─░──────────────────░───────░───────░───────░─»
      └──────────────────┘ ░                  ░       ░       ░       ░ »
«     ┌───────────────────┐ ░ 
«q_0: ┤ U(2π/5,2π/5,8π/5) ├─░─
«     └───────────────────┘ ░ 
«q_1: ──────────────────────░─
«                           ░ 
«q_2: ──────────■───────────░─
«             ┌─┴─┐         ░ 
«q_3: ────────┤ X ├─────────░─
«             └───┘         ░ 
target gate: {'name': 'cx', 'qubits': [2, 3], 'params':

The code generates 4 paths. Each path has at most 2 steps. A step is represented as "gate type,qubits-dependency-gate type,qubits". For example, "u,4-parallel-u,0-parallel-u,8" means that a U gate on qubit 4 is executed in parallel with U gates on qubits 0 and 8. 

## Construction of Path Table

For a gate that requires vectorization, we compare it with a path table. The path table is off-line generated by applying random walks to a circuit dataset. To limits the size of the table, the table is usually hardware-specific.

In [3]:
# define the information of the quantum device
n_qubits = 6
backend = GridBackend(2, 3)

# generate a dataset including varous random circuits
circuit_dataset = random_circuits(backend, n_circuits=100, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

# apply random work to consturct the vectorization model with a path table
n_steps = 1
n_walks = 100
up_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend, decay= .5)
up_model.train(circuit_dataset, multi_process=False)

print('length of the path table is', len(up_model.pathtable))

100%|██████████| 102/102 [00:00<00:00, 228.72it/s]


length of the path table is 445


# Gate Vectorization

As mentioned above, the vectorization of a gate is performed by comparing the generated paths with a path table. In JanusQ, we provide a api to do this. Below is a example of it.

In [4]:
# generate a circuit
circuit = random_circuit(backend, 10, .5, False)

# choose a target gate
gate = random.choice(circuit.gates)

# vectorization
vec = up_model.vectorize(circuit, [gate])[0]
print('vector is', vec)

vector is [0.5 1.  0.5 0.  0.5 0.  0.5 0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0. ]


The indexes of the non-zero elements in the vector is same to the indexes of the generated paths in the path table, which is verified by following codes.

In [5]:
indexes = np.argwhere(vec > 0).flatten()
generated_paths = walk_from_gate(circuit, gate, 100, 1, backend.adjlist)
device = extract_device(gate)

print(list(indexes), '=', up_model.indices_of_paths(device, generated_paths))

[0, 1, 2, 4, 6, 7, 9] = [0, 1, 2, 4, 6, 7, 9]


## Sub-circuit Reconstruction
The vectorization of QuCT also allows the reconstruction of the sub-circuit around the gate by its vector.

In [6]:
circuit = up_model.reconstruct(device, vec)
print(circuit)

      ┌─────────────────────────┐  ░       ░ 
q_0: ─┤ U(17.158,17.142,10.548) ├──░───────░─
      └┬───────────────────────┬┘  ░       ░ 
q_1: ──┤ U(4.064,1.163,1.3467) ├───░───────░─
      ┌┴───────────────────────┴─┐ ░       ░ 
q_2: ─┤ U(2.1857,0.21919,1.4346) ├─░───────░─
     ┌┴──────────────────────────┤ ░       ░ 
q_3: ┤ U(2.8992,3.7818,0.082721) ├─░───────░─
     └┬─────────────────────────┬┘ ░       ░ 
q_4: ─┤ U(5.0598,3.1813,2.3751) ├──░───■───░─
      ├─────────────────────────┤  ░ ┌─┴─┐ ░ 
q_5: ─┤ U(5.7515,4.6988,4.7192) ├──░─┤ X ├─░─
      └─────────────────────────┘  ░ └───┘ ░ 
